In [1]:
import numpy as np
import scipy.constants as const
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
import math
from scipy.integrate import quad
import pandas as pd
import config
from scipy import optimize

In [2]:
userHome = config.userHome()
outdir = userHome + "/projects/cs/data/raw/ext/n2/itikawa/"

In [3]:
pi = const.pi
e = const.e
e0= const.epsilon_0

In [4]:
eV_per_J = const.physical_constants["joule-electron volt relationship"][0]
cm_per_m = 100.0

In [5]:
def Gv(v, we, wexe, weye, weze=0, weae=0):
    return we*(v+1/2) - wexe*(v+1/2)**2 + weye*(v+1/2)**3 + weze*(v+1/2)**4 + weae*(v+1/2)**5

def Te(v, T0, we, wexe, weye, weze=0, weae=0):
    return T0 - we/2 + wexe/4 - weye/8 - weze/16 - weae/32

def Tv(v, T0, we, wexe, weye, weze=0, weae=0):
    return Te(v, T0, we, wexe, weye, weze, weae) + Gv(v, we, wexe, weye, weze, weae)

def Bv(v, Be, ae, ge=0, de=0, ee=0):
    return Be - ae*(v+1/2) - ge*(v+1/2)**2 + de*(v+1/2)**3 + ee*(v+1/2)**4

def fIntegrand(vp, v, vibConstants):
    (T0, we, wexe, weye, weze, weae) = vibConstants
    return 1/math.sqrt(Gv(v,we,wexe,weye,weze)-Gv(vp,we,wexe,weye,weze))

def gIntegrand(vp, v, vibConstants, rotConstants):
    (T0, we, wexe, weye, weze, weae) = vibConstants
    (Be, ae, ge, de, ee) = rotConstants
    return Bv(v,Be,ae,ge)/math.sqrt(Gv(v,we,wexe,weye,weze)-Gv(vp,we,wexe,weye,weze))

In [6]:
statesN2 = ["N2(X1Sigmag+)","N2(A3Sigmau+)","N2(B3Pig)","N2(W3Deltau)","N2(Bp3Sigmau-)",
            "N2(ap1Sigmau-)","N2(a1Pig)","N2(w1Deltau)","N2(C3Piu)","N2(E3Sigmag+)","N2(D3Sigmau+)"]
statesN2p = ["N2+(X2Sigmag+)","N2+(A2Piu)","N2+(B2Sigmau+)","N2+(C2Sigmau+)"]

vibConstantsN2 = [[0.0, 2358.57, 14.324, -2.26E-3, -2.4E-4, 0],
                 [49754.8, 1460.48, 13.775, -1.175E-2, 1.41E-4, -7.29E-5],
                 [59306.8, 1734.38, 14.558, 1.40E-2, -1.13E-3, 0],
                 [59380.2, 1506.53, 12.575, 3.09E-2, -7.1E-4, 0],
                 [65851.3, 1516.88, 12.181, 4.19E-2, -7.3E-4, 0],
                 [67739.3, 1530.25, 12.075, 4.13E-2, -2.9E-4, 0],
                 [68951.2, 1694.21, 13.949, 7.94E-3, 2.9E-4, 0],
                 [71698.4, 1559.50, 12.008, 4.54E-2, 0, 0],
                 [88977.9, 2047.18, 28.445, 2.0883, -5.350E-1, 0],
                 [95774.5, 2218, 16.3, -2.7E-2, -2.6E-3, 0],
                 [103570.9, 2207, 16.3, -2.7E-2, -2.6E-3, 0]]

vibConstantsN2p = [[125667.5, 2207.37, 16.302, -2.67E-3, -2.61E-3, 3.7E-5],
                  [134683.1, 1903.51, 15.029, 2.03E-3, 0, 0],
                  [151233.5, 2420.83, 23.851, -0.3587, -6.192E-2, 0],
                  [190209.5, 2071.5, 9.29, -0.43, 0, 0]]

rotConstantsN2 = [[1.99824, 1.7318E-2, -3.3E-5, 0, 0],
                 [1.45499, 1.8385E-2, 1.24E-5, -6.7E-6, 0],
                 [1.63802, 1.8302E-2, -8.4E-6, -3.4E-6, 0],
                 [1.47021, 1.6997E-2, -1.01E-5, 3.3E-7, 0],
                 [1.4731, 1.668E-2, 1.84E-5, -4.5E-7, 0],
                 [1.4799, 1.657E-2, 2.41E-5, 0, 0],
                 [1.6169, 1.793E-2, -2.93E-5, 0, 0],
                 [1.4963, 1.63E-2, 0, 0, 0],
                 [1.8247, 1.868E-2, -2.28E-3, 7.33E-4, -1.5E-4],
                 [1.9368, 1.90E-2, -1.9E-4, 0, 0],
                 [1.9705, 1.90E-2, -1.9E-4, 0, 0]]

rotConstantsN2p = [[1.93177, 1.900E-2, -1.91E-5, -5.00E-6, 4.6E-8],
                  [1.7442, 1.838E-2, -1.76E-4, 4.4E-6, 0],
                  [2.0845, 2.132E-2, -8.5E-4, 0, 0],
                  [1.5114, 1.10E-3, -8.2E-4, 0, 0]]

In [38]:
math.ceil(22/2)

11

In [7]:
mu=1.16294E-26
wavenumber_per_eV = 8065.6

In [47]:
def f(v, vibConstants):
    if v > 10:
        v_split=math.ceil(v/2)
        I_lower = quad(fIntegrand,-1/2,v_split,args=(v_split, vibConstants))
        I_upper = quad(fIntegrand,v_split,v,args=(v, vibConstants))
        I = I_lower + I_upper
    else:
        I = quad(fIntegrand,-1/2,v,args=(v, vibConstants))
    return 1/(2*const.pi*math.sqrt(2*mu*const.speed_of_light/const.Planck)) * I[0] * 1E8

def g(v, vibConstants, rotConstants):
    if v > 10:
        v_split=math.ceil(v/2)
        I_lower = quad(gIntegrand,-1/2,v_split,args=(v_split, vibConstants, rotConstants))
        I_upper = quad(gIntegrand,v_split,v,args=(v, vibConstants, rotConstants))
        I = I_lower + I_upper
    else:
        I = quad(gIntegrand,-1/2,v,args=(v, vibConstants, rotConstants))
    return 2*const.pi*math.sqrt(2*mu*const.speed_of_light/const.Planck) * I[0] * 1E-8

In [48]:
def r(v, vibConstants, rotConstants):
    fv=f(v, vibConstants)
    gv=g(v, vibConstants, rotConstants)
    return [math.sqrt(fv**2 + fv/gv)-fv, math.sqrt(fv**2 + fv/gv)+fv]

In [49]:
def Jv(vMaxN2, vMaxN2p, stateN2, stateN2p):

    N2pPES = []
    (T0, we, wexe, weye, weze, weae) = vibConstantsN2p[stateN2p]

    for vp in range(vMaxN2p+1):
        #print(vp)
        rvp=r(vp, vibConstantsN2p[stateN2p], rotConstantsN2p[stateN2p])
        #print(rvp)
        Tvp=Tv(vp, T0, we, wexe, weye, weze, weae)/wavenumber_per_eV
        #print(Tvp)
        N2pPES.insert(0, [rvp[0], Tvp])
        N2pPES.append([rvp[1], Tvp])
        #print(N2pPES[0])
    
    N2pPES = np.asarray(N2pPES)
    
    print(N2pPES[:,0])
    print(N2pPES[:,1])
    
    I_N2pPES = interpolate.interp1d(N2pPES[:,0], N2pPES[:,1], kind='quadratic')

    N2PES = []
    JvArray = np.empty([vMaxN2+1,2])
        
    (T0, we, wexe, weye, weze, weae) = vibConstantsN2[stateN2]
    for v in range(len(JvArray)):
        rv=r(v, vibConstantsN2[stateN2], rotConstantsN2[stateN2])
        Tvv=Tv(v, T0, we, wexe, weye, weze, weae)/wavenumber_per_eV
        N2PES.insert(0, [rv[0], Tvv])
        N2PES.append([rv[1], Tvv])
        JvArray[v] = I_N2pPES(rv)-Tvv

    N2PES = np.asarray(N2PES)
        
    return (N2PES, N2pPES, JvArray)

In [50]:
(N2PES, N2pPES, Jvv) = Jv(40,35,0,0)

[0.09464242 0.0948794  0.0948794  0.09512692 0.09512692 0.09538829
 0.09538829 0.09566701 0.09566701 0.09596683 0.09596683 0.09629188
 0.09629188 0.09664689 0.09664689 0.09703733 0.09703733 0.09746979
 0.09746979 0.09795243 0.09795243 0.09849567 0.09849567 0.09911334
 0.09911334 0.09703733 0.09746979 0.09795243 0.09849567 0.09911334
 0.09982474 0.10065831 0.10165953 0.10291009 0.10458906 0.10731515
 0.11668413 0.12091824 0.12412512 0.12692408 0.12949484 0.13191982
 0.13424531 0.13650061 0.13870596 0.14087628 0.14302312 0.13424531
 0.13424531 0.13650061 0.13650061 0.13870596 0.13870596 0.14087628
 0.14087628 0.14302312 0.14302312 0.14515593 0.14515593 0.14728267
 0.14728267 0.14941041 0.14941041 0.15154556 0.15154556 0.15369417
 0.15369417 0.15586213 0.15586213 0.15805529 0.15805529 0.16027963]
[22.342583   22.23267925 22.11781112 21.99792641 21.87298357 21.74295121
 21.60780747 21.46753956 21.32214314 21.17162178 21.01598644 20.85525491
 20.68945122 20.51860515 20.34275164 20.16193024 

ValueError: Expect x to be a 1-D sorted array_like.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

x = list(range(len(Jvv)))
y = Jvv[:,0]
ax1.scatter(x, y, s=10, c='b', marker="s", label='Jvv[:,0]')
y = Jvv[:,1]
ax1.scatter(x,y, s=10, c='r', marker="o", label='Jvv[:,1]')
plt.legend(loc='upper right');
plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

x,y = N2pPES.T
ax1.scatter(x, y, s=10, c='b', marker="s", label='N2pPES')
x,y = N2PES.T
ax1.scatter(x,y, s=10, c='r', marker="o", label='N2PES')
plt.legend(loc='lower right');
plt.show()

In [ ]:
def f_eqn15(x: float):
    "Equation 15 of Kosarim, et al., for all electrons."
    "return (6*(x-1))/(pi*(x+1.5)*(x+9))"
    return (6*(x-1))/(pi*(x+10)*(x+2))

In [ ]:
def sigma_ion(f, n: int, E: float, J_v_vp: float, J_v_vpp: float):
    """Compute cross-sections (10^-17 cm^2) of ionization transitions for 
    Tables 2-4 of Kosarim, et al."""
    return (cm_per_m**2 * eV_per_J**2)/(1.0E-17)  * (1./(4.*pi*e0)**2) * \
        ((n*e**4)/2.) * ( f(E/J_v_vp)/(J_v_vp**2) + f(E/J_v_vpp)/(J_v_vpp**2) )

In [ ]:
datafiles = [outdir+"ionization_N2+_itikawa2006_15",
             outdir+"ionization_N2+_itikawa2006_16",
             outdir+"ionization_N2+_itikawa2006_17"]
sigma_total=[]

for datafile in datafiles:
    df = pd.read_csv(datafile, sep="\t")   # read .tsv file into memory
    sigma_total.append(df.values[:,[1,2]])  # access the numpy array containing values

sigma_total=np.concatenate((sigma_total[0],
                            sigma_total[1],
                            sigma_total[2]), axis=0)

In [ ]:
N2_ionization = 15.581 # best value of the ionization energy of N2

In [ ]:
sigma_total_reduced = np.column_stack((sigma_total[:,0]/N2_ionization, 
                                       (N2_ionization**2)/( (cm_per_m**2 * eV_per_J**2)/(1.0E-17)  * (1./(4.*pi*e0)**2) * ((2*e**4)) ) * sigma_total[:,1]))

In [ ]:
sigma_total_reduced

In [ ]:
# Fit the first set
fitfunc = lambda p, x: p[0]*(x-p[1])/(pi*(x+p[2])*(x+p[3])) # Target function
errfunc = lambda p, x, y: fitfunc(p, x) - y # Distance to the target function
p0 = [6.0, 1.0, 10.0, 2.0] # Initial guess for the parameters
p1, success = optimize.leastsq(errfunc, p0[:], args=(sigma_total_reduced[:,0], sigma_total_reduced[:,1]))

In [ ]:
p1

In [ ]:
success

In [ ]:
def f_eqn_fit(x: float):
    "Fit total ionization data"
    return (p1[0]*(x-p1[1]))/(pi*(x+p1[2])*(x+p1[3]))

In [ ]:
w = 10
h = 6
d = 70
plt.figure(figsize=(w, h), dpi=d)

x=sigma_total_reduced[:,0]
y=sigma_total_reduced[:,1]

x = np.linspace(min(sigma_total_reduced[:,0]), max(sigma_total_reduced[:,0]), 100)
plt.plot(sigma_total_reduced[:,0], sigma_total_reduced[:,1], "ro", x, fitfunc(p1, x), "r-") # Plot of the data and the fit

# Legend the plot
plt.title("Ionization reduced cross-section for N$_2$")
plt.xlabel("E/J")
plt.ylabel("f(E/J)")
plt.legend(('$\sigma$', 'f(E/J)'))

ax = plt.axes()

plt.show()

In [ ]:
vMat=np.repeat(np.array([np.arange(len(Jvv))]),len(Jvv),axis=0).T
EMat=np.repeat(np.array([np.logspace(1.3, 2.3, num=len(Jvv))]),len(Jvv),axis=0)
sigma = np.empty([len(Jvv), len(EMat[0])])
for v in vMat[:,0]:
    sigma[v] = sigma_ion(f_eqn_fit, 2, EMat[0], Jvv[v,0], Jvv[v,1])


In [ ]:
vMat[:,0]

In [ ]:
EMat[0]

In [ ]:
sigma[0]

In [ ]:
sigma[40]

In [ ]:
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm

fig = plt.figure()
ax = fig.gca(projection='3d')
X=EMat
Y=vMat
Z=sigma

# Plot the 3D surface
ax.plot_surface(X, Y, Z, rstride=8, cstride=8, alpha=0.3)

# Plot projections of the contours for each dimension.  By choosing offsets
# that match the appropriate axes limits, the projected contours will sit on
# the 'walls' of the graph
cset = ax.contour(X, Y, Z, zdir='z', offset=0.2, cmap=cm.coolwarm)
cset = ax.contour(X, Y, Z, zdir='x', offset=min(X[0]), cmap=cm.coolwarm)
cset = ax.contour(X, Y, Z, zdir='y', offset=max(Y[:,0]), cmap=cm.coolwarm)

ax.set_xlim(min(X[0]),max(X[0]))
ax.set_ylim(min(Y[:,0]),max(Y[:,0]))
ax.set_zlim(0.2,3.0)

ax.set_xlabel('E')
ax.set_ylabel('v')
ax.set_zlabel('sigma')

plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

x=EMat[0]
y=sigma[0]
ax1.scatter(x, y, s=10, c='b', marker="s", label='v=0')
y=sigma[20]
ax1.scatter(x,y, s=10, c='r', marker="o", label='v=20')
y=sigma[40]
ax1.scatter(x,y, s=10, c='g', marker="d", label='v=40')
plt.legend(loc='lower right');
plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

x=vMat[:,0]
y=sigma[:,0]
ax1.scatter(x, y, s=10, c='b', marker="s", label='E='+str(round(EMat[0,0])))
y=sigma[:,10]
ax1.scatter(x,y, s=10, c='r', marker="o", label='E='+str(round(EMat[0,10])))
y=sigma[:,20]
ax1.scatter(x,y, s=10, c='g', marker="d", label='E='+str(round(EMat[0,20])))
y=sigma[:,40]
ax1.scatter(x,y, s=10, c='black', marker="+", label='E='+str(round(EMat[0,40])))
plt.legend(loc='upper left');
plt.show()